## Preparing Data

* TorchText를 이용해서 pre-Processing이 필요.
* spacy를 이용해서 data를 tokenization함.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

## Torchtext Contents

#### 사용방법

1. 필드지정(Create Field)
2. 데이터 세트 만들기(Create Datasets)
3. 단어장 생성(Build vocabulary)
4. 데이터 로더 만들기(Create Data Loader)

* 테스트


In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#### Requriments

* "en" : English
* "de" : German

```python -m spacy download en ```

```python -m spacy download de ```


In [3]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

In [4]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
SRC = Field(tokenize = tokenize_de,
           init_token = '<sos>',
           eos_token = '<eos>',
           lower = True)

TRG = Field(tokenize = tokenize_en,
           init_token = '<sos>',
           eos_token = '<eos>',
           lower = True)

In [6]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

In [7]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [8]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [9]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Building the Seq2Seq Model

In [11]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [13]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

## Layer 구성

이 구현을 위해서는 hidden layer와 cell dimension이 Encoder와 Decoder에서 동일해야 한다.
Seq2Seq Model에서 반드시 동일한 수의 레이어 또는 동일한 숨겨진 치수 크기가 필요하지는 않음.
다른 수의 레이어를 갖는 것과 같은 작업을 수행 한 경우 이를 처리하는 방법을 결정 해야 한다.
Ex) Encoder 2개와 Decoder 1개만 있는 경우, Decoder가 출력한 두 개의 Context Vector를 평균화 할 것인지, Linear Layer를 모두 통과할 것인지, 최상위 Layer의 Context Vector만 사용할 것인지 등.

## 자기회기 속성

Seq2Seq의 훈련 방식과 추론 방식의 차이는 근본적으로 자기 회기 autoregression(AR)라는 속성 때문에 생김. 자기 회기란 과거의 자신의 값을 참조하여 현재의 값을 추론하는 특징을 가리킴.과거에 잘못 예측 했을 경우, 시간이 지날 수록 더 큰 잘못된 예측을 할 가능성을 야기 함. 또한 과거의 결괏값에 따라 문장의 구성이 바뀔 뿐만 아니라, 예측 문장의 길이마저도 바뀔 수 있음. 학습 과정에서 이미 정답을 알고 있고, 현재 모델의 예측 값과 정답과의 차이를 통해 학습하므로, 우리는 자기 회기 속성을 유지한채 훈련 할 수 없음.

## Teacher Forcing

Teacher Forcing : RNN Model에서 출력을 다시 입력으로 넣는 구조에서 학습 시킬때와 예측할 때가 달라 질 수 있음, 예측을 할 때는 바로 출력을 입력으로 넣음녀 되지만, 학습을 시킬때는 뒤로 갈 수록 실제 데이터와 입력이 달라져 버리는 경우가 생김.
학습시킬때는 실제 데이터를 입력에 넣어 주는 방법을 사용하게 되는데 이 기법을 Teacher Forcing 기법이라고 함.

참조 : https://kh-kim.gitbook.io/natural-language-processing-with-pytorch/00-cover-9/05-teacher-forcing

## 학습하는 동안 Teacher forcing을 사용하지 않으려면

일부 환경에서는 완전한 입력-목표 시퀀스 쌍을 버퍼링 할 수 없듯이(매우 긴 시퀀스는 online 학습)이는 전체 목표 시퀀스로 접근 할 수 없기 때문에 Teacher forcing을 사용할 수 없음.
이 경우 decoder의 예측값을 입력으로 재입력하여 학습을 실행 할 수 있음(추론)

참조 : https://tykimos.github.io/2018/09/14/ten-minute_introduction_to_sequence-to-sequence_learning_in_Keras/

## Teacher_forcing_ratio

teacher_forcing_ratio와 같은 확률로 다음 시간 단계 동안 디코더에 대한 입력으로 true면 진실(예측값과 정답이 같으면), 다음 token을 사용, 그러나 1-teacher_forcing_ratio에서는 시퀀스의 실제 다음 토큰과 일치하지 않아도 모델이 모델의 다음 입력으로 예측 한 token을 사용함.

참조 : https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb

In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        hidden, cell = self.encoder(src)
        
        #first input to tteh decoder is the : <sos>
        
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            # 가장 높은 예측 값을 predictions 결정
            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1
        
        return outputs

In [16]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [17]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,899,013 trainable parameters


In [20]:
optimizer = optim.Adam(model.parameters())

In [21]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [22]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [23]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [24]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### 언어 평가 지표 방법 : Perplexity

펄플렉서티(perplexity)는 언어 모델을 평가하기 위한 내부 평가 지표입니다. 보통 줄여서 PPL이 라고 표현합니다. 왜 perplexity라는 용어를 사용했을까요? 영어에서 'perplexed'는 '헷갈리는'과 유사한 의미를 가집니다. 그러니까 여기서 PPL은 '헷갈리는 정도'로 이해합시다. PPL를 처음 배울때 다소 낯설게 느껴질 수 있는 점이 있다면, PPL은 수치가 높으면 좋은 성능을 의미하는 것이 아니라, '낮을수록' 언어 모델의 성능이 좋다는 것을 의미한다는 점입니다.

PPL은 단어의 수로 정규화(normalization) 된 테스트 데이터에 대한 확률의 역수입니다. PPL을 최소화한다는 것은 문장의 확률을 최대화하는 것과 같습니다. 문장 W의 길이가 N이라고 하였을 때의 PPL은 다음과 같습니다.

PPL(W)=P(w1,w2,w3,...,wN)−1N=1P(w1,w2,w3,...,wN)−−−−−−−−−−−−−−−−−−√N
문장의 확률에 체인룰(chain rule)을 적용하면 아래와 같습니다.

PPL(W)=1P(w1,w2,w3,...,wN)−−−−−−−−−−−−−−−−−−√N=1∏Ni=1P(wi|w1,w2,...,wi−1)−−−−−−−−−−−−−−−−−−−−−−√N
여기에 n-gram을 적용해볼 수도 있습니다. 예를 들어 bigram 언어 모델의 경우에는 식이 아래와 같습니다.

PPL(W)=1∏Ni=1P(wi|wi−1)−−−−−−−−−−−−−−√N



reference :https://wikidocs.net/21697

In [26]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 37s
	Train Loss: 3.958 | Train PPL:  52.369
	 Val. Loss: 4.486 |  Val. PPL:  88.736
Epoch: 02 | Time: 0m 36s
	Train Loss: 3.797 | Train PPL:  44.581
	 Val. Loss: 4.303 |  Val. PPL:  73.954
Epoch: 03 | Time: 0m 37s
	Train Loss: 3.632 | Train PPL:  37.791
	 Val. Loss: 4.209 |  Val. PPL:  67.260
Epoch: 04 | Time: 0m 37s
	Train Loss: 3.504 | Train PPL:  33.248
	 Val. Loss: 4.099 |  Val. PPL:  60.286
Epoch: 05 | Time: 0m 36s
	Train Loss: 3.345 | Train PPL:  28.369
	 Val. Loss: 4.055 |  Val. PPL:  57.696
Epoch: 06 | Time: 0m 37s
	Train Loss: 3.218 | Train PPL:  24.984
	 Val. Loss: 3.952 |  Val. PPL:  52.062
Epoch: 07 | Time: 0m 37s
	Train Loss: 3.092 | Train PPL:  22.014
	 Val. Loss: 3.905 |  Val. PPL:  49.663
Epoch: 08 | Time: 0m 37s
	Train Loss: 2.992 | Train PPL:  19.922
	 Val. Loss: 3.789 |  Val. PPL:  44.223
Epoch: 09 | Time: 0m 37s
	Train Loss: 2.881 | Train PPL:  17.838
	 Val. Loss: 3.828 |  Val. PPL:  45.980
Epoch: 10 | Time: 0m 37s
	Train Loss: 2.798 | Train PPL

Epoch: 80 | Time: 0m 38s
	Train Loss: 0.528 | Train PPL:   1.696
	 Val. Loss: 5.235 |  Val. PPL: 187.746
Epoch: 81 | Time: 0m 38s
	Train Loss: 0.522 | Train PPL:   1.686
	 Val. Loss: 5.314 |  Val. PPL: 203.131
Epoch: 82 | Time: 0m 37s
	Train Loss: 0.518 | Train PPL:   1.679
	 Val. Loss: 5.312 |  Val. PPL: 202.746
Epoch: 83 | Time: 0m 37s
	Train Loss: 0.502 | Train PPL:   1.652
	 Val. Loss: 5.396 |  Val. PPL: 220.416
Epoch: 84 | Time: 0m 37s
	Train Loss: 0.507 | Train PPL:   1.660
	 Val. Loss: 5.360 |  Val. PPL: 212.724
Epoch: 85 | Time: 0m 37s
	Train Loss: 0.482 | Train PPL:   1.619
	 Val. Loss: 5.403 |  Val. PPL: 222.096
Epoch: 86 | Time: 0m 37s
	Train Loss: 0.481 | Train PPL:   1.617
	 Val. Loss: 5.434 |  Val. PPL: 229.135
Epoch: 87 | Time: 0m 38s
	Train Loss: 0.476 | Train PPL:   1.609
	 Val. Loss: 5.473 |  Val. PPL: 238.085
Epoch: 88 | Time: 0m 38s
	Train Loss: 0.476 | Train PPL:   1.610
	 Val. Loss: 5.436 |  Val. PPL: 229.504
Epoch: 89 | Time: 0m 37s
	Train Loss: 0.454 | Train PPL

In [27]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.657 | Test PPL:  38.735 |


Reference : https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb